# Introduction to Machine Learning 3
Na-Rae Han, 10/19/2019

## Beyond Naive Bayes: other ML methods?
- We will try **SVM** (**Support Vector Machine**). 
- Textbook section on SVM: https://jakevdp.github.io/PythonDataScienceHandbook/05.07-support-vector-machines.html

## Feature engineering, categorical data
- Feature engineering
- Handling categorical data
- Testbook section: https://jakevdp.github.io/PythonDataScienceHandbook/05.04-feature-engineering.html
- A better explanation here: https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f
- And here: http://pbpython.com/categorical-encoding.html

In [ ]:
# Turns on/off pretty printing 
%pprint

# Every returned Out[] is displayed, not just the last one. 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn               # sklearn is the ML package we will use
import seaborn as sns        # seaborn graphical package

### Data preparation step

In [ ]:
from sklearn.datasets import fetch_20newsgroups

# We will use the same 4 categories
cats = ['talk.religion.misc', 'soc.religion.christian', 'sci.space', 'comp.graphics']

train = fetch_20newsgroups(subset='train', categories=cats)
test = fetch_20newsgroups(subset='test', categories=cats)

In [ ]:
type(train)

In [ ]:
dir(train)

In [ ]:
train.data[5]

In [ ]:
train.target
train.target[5]
train.target_names

In [ ]:
len(train.data)
len(test.data)

In [ ]:
# TfidfVectorizer is essentially CountVectorizer + TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

# Ignore words found in over 50% of documents, ignore words found in just 1 document. 
# 1000 most frequent words, remove stop words. 
vectorizer = TfidfVectorizer(max_df=0.5, min_df=2, max_features=1000, stop_words='english')
X_train = vectorizer.fit_transform(train.data)
y_train = train.target
X_test = vectorizer.transform(test.data)
y_test = test.target

# Cumbersome. Alternative: using pipeline. 

### Time to build: Support Vector Classifier

In [ ]:
from sklearn.svm import SVC # "Support vector classifier"
model = SVC(kernel='linear', C=1E10)
model.fit(X_train, y_train)

In [ ]:
labels = model.predict(X_test)
labels

In [ ]:
y_test[:20]
labels[:20]

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, labels)
cm

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, labels)

In [ ]:
sns.heatmap(cm.T, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=train.target_names, yticklabels=train.target_names)
plt.xlabel('true label')
plt.ylabel('predicted label')
plt.show()

### Is this better than Naive Bayes?
What other classification algorithms are there?
- Logistic Regression
- K-Nearst Neighbors
- Decision Tree, Random Forest https://jakevdp.github.io/PythonDataScienceHandbook/05.08-random-forests.html
- Deep Learning

## Categorical data
Margaret's ASL class data as an example:
https://github.com/Data-Science-for-Linguists/Class-Practice-Repo/blob/master/todo3/asl-background-cut_margaret.csv

(Make sure to click on "Raw" button to get to raw CSV file format)

In [ ]:
url = 'https://raw.githubusercontent.com/Data-Science-for-Linguists/Class-Practice-Repo/master/todo3/asl-background-cut_margaret.csv'
df = pd.read_csv(url)

In [ ]:
df.head()
df.tail()

### Questions: 
- Which columns are truly categorical? 
- How to best convert them to numerical values? 

### Back to ETS data

In [ ]:
ets_df = pd.read_csv('ets_df_final.csv', index_col=0)
ets_df.head()

### Questions:
- Which columns are truly categorical?

`L1`
- As classification target?
- As classificatoin feature?

`Score`
- As classification target?
- As classification feature? 

### Pseudo-categorical data must be converted into numerical values
`Score` should be converted into corresponding numbers, both as target and feature. 
- Low: 1
- Medium: 2
- High: 3

In predicting `Score`, you will be building a **REGRESSION model**, not a classification model. 
- Predicted values will come out as 0.8, 1.2, 2.8, 3.1 etc.
- The continuous numeric values will then have to be binned into discrete categories such as 1, 2, 3 or 'Low', 'Medium', 'High'. 

### Truly categorical data
- Can be used as classification target as is. You will be building a true **CLASSIFICATION model**. 
- When used as *features*, they must be DECOMPOSED into a set of binary, independent features.
   1. Option 1: Encode value labels as integers: map 'Korean' 0, 'German' to 1, 'Japanese' to 3 (through `LabelEncoder`)
      - This step alone isn't enough. (WHY?)
      - You must then encode map integer labels to a set of binary-valued columns (through `OneHotEncoder`)
   2. Option 2. Transform the categorical column into a set of **dummy variables** (through `pandas.get_dummes`) 
      

#### Option 1: LabelEncoder + OneHotEncoder

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [ ]:
mini = ets_df[:10][['Prompt', 'L1', 'Score']]
mini

In [ ]:
lenc = LabelEncoder()
lenc.fit(mini['L1'])
lenc.transform(mini['L1'])

In [ ]:
mini['L1'] = lenc.transform(mini['L1'])
mini

In [ ]:
ohenc = OneHotEncoder()
ohenc.fit_transform(mini[['L1']]).toarray()
#ohenc.fit(mini['L1'].values.reshape(-1,1))
#ohenc.transform(mini['L1'].values.reshape(-1,1))

#### Option 2: Use `pandas.get_dummies`

In [ ]:
mini = ets_df[:10][['Prompt', 'L1', 'Score']]
mini

In [ ]:
pd.get_dummies(mini, columns=['L1'])